In [1]:
import numpy as np
import random
import math
import multiprocess as mp
import talib as ta
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime,date
from gga import GGA
from island_ga import IslandGGA 

In [2]:
data=pd.read_csv('HistoricalQuotes.csv')
data.head()

,Date,Close/Last,Volume,Open,High,Low
0,02/28/2020,$273.36,106721200,$257.26,$278.41,$256.37
1,02/27/2020,$273.52,80151380,$281.1,$286,$272.96
2,02/26/2020,$292.65,49678430,$286.53,$297.88,$286.5
3,02/25/2020,$288.08,57668360,$300.95,$302.53,$286.13
4,02/24/2020,$298.18,55548830,$297.26,$304.18,$289.23


In [3]:
#rename column to remove spaces in column names
aapl =data.rename(columns={' Close/Last': 'close', ' Volume': 'volume',
                           ' Open':'open',' High':'high',' Low':'low'})
                        

#change date string into date format and sort the dataframe in ascending order
aapl[['Date']] = aapl[['Date']].applymap(lambda x: datetime.strptime(x,'%m/%d/%Y').date())
aapl =aapl.sort_values(by='Date')
#change dataframe dtypes to float
cols = [i for i in aapl if i not in ['Date','volume']]
def to_float(x):
    x = x.strip()
    return float(x[1:])

aapl[cols]=aapl[cols].applymap(to_float)

#Create Signals using Talib library
aapl['5EMA'] = ta.SMA(np.array(aapl['close']),5)
aapl['20EMA'] = ta.EMA(np.array(aapl['close']), timeperiod = 20)
aapl['RSI'] = ta.RSI(np.array(aapl['close']), timeperiod = 14)
aapl['WILLR'] = ta.WILLR(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)
aapl['MOM'] = ta.MOM(np.array(aapl['close']), timeperiod=5)
aapl['CCI'] = ta.CCI(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)
aapl['SLOWK'],aapl['SLOWD'] = ta.STOCH(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), fastk_period=14, slowk_period=3, slowd_period=3)
aapl['MACD'],aapl['MACDSIGNAL'],aapl['MACDHIST'] = ta.MACD(np.array(aapl['close']), fastperiod=12, slowperiod=26, signalperiod=9)
aapl['DMI'] = ta.DX(np.array(aapl['high']), np.array(aapl['low']), np.array(aapl['close']), timeperiod=14)   

In [4]:
#  """Based on parameter setting adopted in Chen et al (2021) """

def generate_candidate_trading_signals(data):
    """Based on parameter setting adopted in Chen et al (2021) """
    conditions ={'TS1_CONDITIONS':[
                (data['5EMA'] > data['20EMA']),
                (data['5EMA'] < data['20EMA'])],
                 'TS2_CONDITIONS':[
                (data['RSI'] > 30),
                (data['RSI'] < 70),
                ],
                 'TS3_CONDITIONS':[
                (data['WILLR'] < 80),
                (data['WILLR'] > 20),
                ],
                 'TS4_CONDITIONS':[
                (data['MOM'] > 0 ),
                (data['MOM'] <= 0),
                ],
                 'TS5_CONDITIONS': [
                (data['CCI'] > 100 ),
                (data['CCI'] <= 100),
                ],
                 'TS6_CONDITIONS': [
                (data['SLOWK'] > data['SLOWD']) & (data['SLOWD'] < 20),
                (data['SLOWK'] < data['SLOWD']) & (data['SLOWD'] > 80)],
                'TS7_CONDITIONS': [
                (data['MACD'] > 0 ),
                (data['MACD'] <= 0)],
                'TS8_CONDITIONS': [
                (data['CCI'] > 100 ),
                (data['CCI'] <= -100)]}

    # create a list of the values we want to assign for each condition 1: buy, 0: sell
    values = [1, 0]

    # create a new column and use np.select to assign values to it using our lists as arguments
    for i in conditions:
        data[i] = np.select(conditions[i], values)
    strategies =list(conditions.keys())
    return data,strategies

In [5]:
data, strategies =generate_candidate_trading_signals(aapl.copy())
num_islands=3
num_iter=20
pSize=20
m_iter=5
N=5
K=3
r_cross=0.8
r_mut= 0.01
r_inv=0.3
n=4
b=4
stop_loss=-0.15
take_profit=0.15
allocated_capital=1000
islandgga = IslandGGA(num_islands,num_iter,data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)

In [6]:
pop= islandgga.init_population()

In [7]:
islandgga.evolve_island_ring()

[[0, 1, 1, 1, 0, 0, 1, 0], [['TS8_CONDITIONS', 'TS2_CONDITIONS', 'TS1_CONDITIONS', 'TS4_CONDITIONS', 'TS1_CONDITIONS', 'TS7_CONDITIONS'], ['TS4_CONDITIONS', 'TS5_CONDITIONS', 'TS3_CONDITIONS', 'TS3_CONDITIONS'], ['TS6_CONDITIONS', 'TS7_CONDITIONS', 'TS2_CONDITIONS']], [0, 1, 1, 0, 1, 1, 1], 59458.651989702754]


In [12]:
m = IslandGGA(num_islands,num_iter,data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)
m.evolve_island_multikuti()

[[1, 1, 0, 0, 0, 1, 1, 1], [['TS2_CONDITIONS', 'TS7_CONDITIONS', 'TS1_CONDITIONS', 'TS4_CONDITIONS'], ['TS6_CONDITIONS', 'TS5_CONDITIONS', 'TS4_CONDITIONS', 'TS7_CONDITIONS', 'TS7_CONDITIONS'], ['TS3_CONDITIONS', 'TS8_CONDITIONS', 'TS8_CONDITIONS', 'TS7_CONDITIONS']], [1, 0, 1, 0, 1, 1, 1], 62151.60890564015]


In [8]:
m = IslandGGA(num_islands,num_iter,data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)
m.evolve_master_slave()

In [9]:
m.globalBest

[[0, 1, 0, 0, 1, 1, 0, 1],
 [['TS1_CONDITIONS', 'TS7_CONDITIONS', 'TS5_CONDITIONS'],
  ['TS8_CONDITIONS', 'TS3_CONDITIONS', 'TS4_CONDITIONS'],
  ['TS6_CONDITIONS', 'TS2_CONDITIONS']],
 [0, 0, 1, 0, 1, 1, 1],
 9091.630081228954]

In [ ]:
# select trading strategy according to SLTP

def generate_trading_signal(data,strategy,stop_loss,take_profit):
    total_strategy_return = 0
    trade_freq = 0
    market_position = 'out'
    max_loss = 0
    for row in data.itertuples(index=False): 
        if market_position == 'out' :
            if row[data.columns.get_loc(strategy)] == 1:
                cost_price = row.close
                market_position = 'in'
        else:
            sell_price = row.close
            trade_return = round(((sell_price - cost_price)/cost_price),2)
            if trade_return <= stop_loss or trade_return >= take_profit: 
                trade_freq +=1
                if trade_return < max_loss:
                    max_loss = trade_return
                total_strategy_return += trade_return
                market_position = 'out'   
            if row[data.columns.get_loc(strategy)] == 0 and trade_return >= take_profit :
                market_position = 'out'
                trade_freq +=1
                if trade_return < max_loss:
                    max_loss = trade_return
                total_strategy_return += trade_return
    avg_return = round(total_strategy_return/trade_freq,2)
    strategy_performance ={strategy: {"avg_return":avg_return,"mdd":max_loss,"frequency":trade_freq}}
    return strategy_performance

def generateSLTP(n,b):
    """Generate bits for SLTP Part"""
    l = n + b
    sltp= [random.randint(0, 1) for _ in range(l)]
    
    return sltp

def binary_to_sltp(sltp,n,b,sl_boundary,tp_boundary):
    """CONVERT SLPT Part to float"""
    
    sl_part,tp_part = sltp[:n],sltp[n:]
    max_sl = sum([np.power(2,i)*1 for i in range(len(sl_part))])
    max_tp = sum([np.power(2,i)*1 for i in range(len(tp_part))])
    sl_part.reverse()
    tp_part.reverse()
    sl  = sum([np.power(2,i)*sl_part[i] for i in range(len(sl_part))])
    tp  = sum([np.power(2,i)*tp_part[i] for i in range(len(tp_part))])
    sl = sl_boundary/max_sl * sl
    tp = tp_boundary/max_tp * tp
    return sl,tp

def strategy_performance(data,strategies,sltp,n,b,stop_loss,take_profit):
    sl, tp = binary_to_sltp(sltp,n,b,stop_loss,take_profit)
    strategy_performance = {}
    for strategy in strategies:
        a = generate_trading_signal(data,strategy,sl,tp)
        
        strategy_performance[strategy]= a[strategy]
    return strategy_performance

def generateGroup(K,strategies):
    """Generate Group and assign TS to groups K"""
    x = strategies.copy()
    random.shuffle(x)
    groups = [[] for k in range(K)]
    while x:
        s = x.pop()
        random_index = random.randrange(K)
        groups[random_index].append(s)
    return groups

def generateWeight(K,num_weight):
    """Generate Weight Part and assign to groups K"""
    weights= [1 for _ in range(num_weight)]
    K = K+1
    for i in range(K):
        random_index = random.randrange(K)
        weights[random_index] = 0
    return weights

def init_population(pSize,n,b,K,num_weight,strategies):
    population =[]
    for i in range(pSize):
        chromosome = [] #c = [[SLTP],[[K],[Weight]]
        chromosome.append(generateSLTP(n,b)) #SLPT PART
        chromosome.append(generateGroup(K,strategies)) # TS & Group Part"
        chromosome.append(generateWeight(K,num_weight)) #Weight Part
        chromosome.append(1) #fitness
        population.append(chromosome)
    
    return population





In [ ]:
 #Fitness Function
    
def get_max_min(p4mc,metric):
    modified ={}
    for i in  p4mc:
        modified[i] = p4mc[i][metric]
    max_key= max(modified, key=modified.get)
    min_key = min(modified, key=modified.get)
    
    return modified[max_key], modified[min_key]


    
def getRisk(p4mc,chromosome):
    all_tsp = 0
    for i in range(len(chromosome[1])):
        
        tsp =[]
        for j in chromosome[1][i]:
            tsp.append(p4mc[j]['mdd'])
        if tsp:
            mdd = min(tsp)
        else:
            mdd =0
        all_tsp += mdd
    all_tsp = all_tsp/len(chromosome[1])
    return all_tsp

def normalisation(p4mc,metric):
    max_value,min_value = get_max_min(p4mc,metric)
    for i in p4mc:
        p4mc[i][metric] = (p4mc[i][metric] - min_value )/ (max_value - min_value)
    return p4mc

def getWeights(weightPart,K):
    w = weightPart.copy()
    L = sum([i for i in w if i == 1])
    K = K+1
    z = {k: [] for k in range(K)}
    for i in range(K):
        while w:
                x = w.pop(0)
                if x == 0:
                    break
                else:

                    z[i].append(x)
    
    return z
    


def getProfit(p4mc,chromosome,allocated_capital,K):
    weights = getWeights(chromosome[2],K)
    w = chromosome[2]
    L = sum([i for i in w if i == 1])
    for i in weights:
        try:
            weights[i]= round(len(weights[i])/L,2)
        except ZeroDivisionError:
            weights[i] = 0
    total = 0
    for i in range(len(chromosome[1])):
        for j in chromosome[1][i]:
            total += p4mc[j]['avg_return']*weights[i+1]*allocated_capital
    return total

def groupBalance(chromosome,N):
    gb = 0
    for group in chromosome[1]:
        try:
            g_result = len(group)/N
        except ZeroDivisionError:
            g_result = 0
        if g_result == 0:
            g =0
        else:
            g = -g_result * np.log(g_result)
        gb += g
    return gb

def weightBalance(chromosome,K):
    gb = 0
    TL = sum([i for i in chromosome[2] if i == 1])
    weights = getWeights(chromosome[2],K)
    
    for i in weights:
        try:
            w = len(weights[i])/TL
        except ZeroDivisionError:
            w = 0
        if w == 0:
            wb = 0  
        else:
            wb = -w * np.log(w)
        if wb:
            gb += wb
    return gb

def fitness_function(chromosome,strategies,n,b,stop_loss,take_profit,allocated_capital,K):
    performance = strategy_performance(data,strategies,chromosome[0],n,b,stop_loss,take_profit)
    p4mc = normalisation(performance,'mdd')
    profit =getProfit(p4mc,chromosome,allocated_capital,K)
    risk = getRisk(p4mc,chromosome)
    
    gb = groupBalance(chromosome,len(strategies))
 
    wb = weightBalance(chromosome,K)
    fitness = profit * risk * np.power(gb,2) * wb
    chromosome[3] = fitness 
    return chromosome

def hamming_distance(chromosome_string1, chromosome_string2): 	
    dist_counter = 0 
    #sltp part
    for n in range(len(chromosome_string1[0])):
        if chromosome_string1[0][n] != chromosome_string2[0][n]:
            dist_counter += 1
    #grouping part
    for n in range(len(chromosome_string1[1])):
        if chromosome_string1[1][n] != chromosome_string2[1][n]:
            dist_counter += 1
    #weight part
    for n in range(len(chromosome_string1[2])):
        if chromosome_string1[2][n] != chromosome_string2[2][n]:
            dist_counter += 1
    return dist_counter

# Select parents

def roulette_wheel_selection(population):
  
    # Computes the totallity of the population fitness
    #population_fitness = sum([chromosome[3] for chromosome in population])
    population_fitness = 0
    for chromosome in population:
        population_fitness += chromosome[3]
    # Computes for each chromosome the probability 
    chromosome_probabilities = []
    for chromosome in population:
        prob = chromosome[3]/population_fitness
        if chromosome[3] != 0:
            chromosome_probabilities.append(chromosome[3]/population_fitness)
        else:
            chromosome_probabilities.append(0)
    #chromosome_probabilities = [chromosome[3]/population_fitness for chromosome in population]
    # Selects one chromosome based on the computed probabilities
    population = np.array(population,dtype=object)
    output = population[np.random.choice(population.shape[0],p=chromosome_probabilities)]
    
    return list(output)#np.random.choice(population, p=chromosome_probabilities)

def selection(population):
    selected = []
    for i in range(len(population)):
        selected.append(roulette_wheel_selection(population))
    return selected

### Crossover
def crossover(parent1,parent2,r_cross):
    child1 = parent1
    child2 = parent2
    # check for recombination
    if random.random() < r_cross:
        # select crossover point that is not on the end of the string
        index = random.randint(1, len(parent1[0])-2)
        # perform crossover on SLTP
        child1[0] = parent2[0][:index] + parent1[0][index:]
        child2[0] = parent1[0][:index] + parent2[0][index:]
        # perform crossover on weight
        index = random.randint(1, len(parent1[2])-2)
        child1[2] = parent2[2][:index] + parent1[2][index:]
        child2[2] = parent1[2][:index] + parent2[2][index:]
    return child1,child2
###Mutation

def mutation(chromosome, r_mut):
    # on SLTP
    for i in range(len(chromosome[0])):
        # check for a mutation
        if random.random() < r_mut:
            # flip the bit
            chromosome[0][i] = 1 - chromosome[0][i]
     # on Weight Part
    for i in range(len(chromosome[2])):
        # check for a mutation
        if random.random() < r_mut:
            # flip the bit 
            chromosome[2][i] = 1 - chromosome[2][i] 
    # on TS part
    e = random.random()
    if e < r_mut:
            grp_idx1 = random.randrange(len(chromosome[1]))
            grp_idx2 = random.randrange(len(chromosome[1]))
            ts_idx = random.randrange(len(chromosome[1][grp_idx1]))
            ts = chromosome[1][grp_idx1][ts_idx]
            chromosome[1][grp_idx2].append(ts)
            
    return chromosome
###Inversion
def inversion(chromosome, r_inv):
    if random.random() < r_inv:
        inverted = chromosome[1][::-1]
        chromosome[1] = inverted
        #chromosome[1][grp_idx2] = ts_1
    return chromosome


In [ ]:
##GGA

def gga(data,K,pSize,strategies,r_cross,r_mut,r_inv,n_iter,n,b,stop_loss,take_profit,allocated_capital):
    num_weight = (K*2) + 1
    population = init_population(pSize,n,b,K,num_weight,strategies)
    best = population[0]
    for j in range(n_iter):
        
        for chromosome in population:
            fitness_function(chromosome,strategies,n,b,stop_loss,take_profit,allocated_capital,K)
        tempPopu  = selection(population)
        children = []
        #Crossover
        for i in range(0, len(tempPopu)-1, 2):

                # get selected parents in pairs
                parent1,parent2 = tempPopu[i],tempPopu[i+1]
                #crossover and mutation and inversion 
                child1,child2 = crossover(parent1,parent2,r_cross)
                child1 = mutation(child1, r_mut)
                child2 = mutation(child2, r_mut)
                child1 = inversion(child1, r_inv)
                child2 = inversion(child2, r_inv)
                children.append(child1)
                children.append(child2)
        population = children
        for chromosone in population:
                if chromosone[3] > best[3]:
                    best = chromosone
    return best
        

In [ ]:
#split data
train = aapl[(aapl['Date'] <= date(2016,12,31))]
test = aapl[(aapl['Date'] > date(2016,12,31))]

In [ ]:
##ISLAND GGA RING TOPOLOGY process implemtation
import time
from multiprocessing import Queue ,Process
#Island GA functions 



def best_chromosomes(population,N,q):
    q.put([i for i in sorted(population, key=lambda x: x[3])[-N:]])

def select_best_chromosomes(population,N):
    return [i for i in sorted(population, key=lambda x: x[3])[-N:]]

def worst_chromosomes(population,N):
    worst = [i for i in sorted(population, key=lambda x: x[3])[:N]]
    
    for i in worst:
        population.remove(i)
    return population

def evolve(island,strategies,n,b,stop_loss,take_profit,allocated_capital,K,r_cross,r_mut,r_inv,q):
    """evolve each island per generation"""
    for chromosome in island:
        fitness_function(chromosome,strategies,n,b,stop_loss,take_profit,allocated_capital,K)
    tempPopu  = selection(island)
    children = []
    #Crossover
    for i in range(0, len(tempPopu)-1, 2):
            # get selected parents in pairs
            parent1,parent2 = tempPopu[i],tempPopu[i+1]
            #crossover and mutation and inversion 
            child1,child2 = crossover(parent1,parent2,r_cross)
            child1 = mutation(child1, r_mut)
            child2 = mutation(child2, r_mut)
            child1 = inversion(child1, r_inv)
            child2 = inversion(child2, r_inv)
            children.append(child1)
            children.append(child2)
    island = children
    q.put(island)   


def island_gga(num_island,num_iter,data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital):
    num_weight = (K*2) + 1
    #intiate population
    islands = []
    for i in range(num_island):
        islands.append(init_population(pSize,n,b,K,num_weight,strategies))
    print("init")
    #evolve
    
    for iteration in range(num_iter):
        processes = []
        result_queues = []
        for j in range(num_island):
            result_queue = mp.Queue()
            process = mp.Process(target=evolve, args=(islands[j], strategies,n,b,stop_loss,take_profit,allocated_capital,K,r_cross,r_mut,r_inv,result_queue))
            process.start()
            processes.append(process)
            result_queues.append(result_queue)
        for process in processes:
            process.join()
        for j in range(num_island):
            islands[j] = result_queues[j].get()

    #migration 
        if iteration % m_iter ==0:
            for j in range(num_island):
                processes = []
                result_queues = []
                send_individuals = select_best_chromosomes(islands[j], N)
                islands[j]=worst_chromosomes(islands[j],N)
                receive_individuals = []
                for k in range(num_island):
                    if k != j:
                        result_queue = mp.Queue()
                        process = mp.Process(target=best_chromosomes, args=(islands[k], N,result_queue))
                        process.start()
                        processes.append(process)
                        result_queues.append(result_queue)
                for process in processes:
                    process.join()
                for result in result_queues:
                    receive_individuals.extend(result.get())
                for individual in receive_individuals:
                    islands[j].append(individual)
    # Return the best individual from each island
    best_individuals = []
    processes = []
    result_queues = []
    for i in range(num_island):
        result_queue =mp.Queue()
        process =mp.Process(target=best_chromosomes, args=(islands[i],1,result_queue))
        process.start()
        processes.append(process)
        result_queues.append(result_queue)
    for process in processes:
        process.join()
    for process in result_queues:
        best_individuals.append(process.get())
    return best_individuals

def main():
    data,strategies =  generate_candidate_trading_signals(aapl.copy())
    num_island = 3 # number of islands
    m_iter =  5 # migration interval
    N = 1 # migration rate
    K = 3 # number of grps
    pSize = 5 # pop size
    r_cross = 0.8 # cross over rate
    r_mut = 0.01 # mutation rate
    r_inv = 0.3 # inversion rate
    num_iter = 10 # number of generations
    n = 4 # number of SL  bits
    b = 4 # number of TP bits
    stop_loss = -0.15
    take_profit = 0.15
    allocated_capital=1000
    results = island_gga(num_island,num_iter,data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)
    
    print(results)
    

if __name__ == "__main__":
   main()
   #print("cool")


        

In [ ]:
##ISLAND GGA RING TOPOLOGY pool implementation
import time
from multiprocessing import Queue ,Process
#Island GA functions 



def best_chromosomes(population,N,q):
    q.put([i for i in sorted(population, key=lambda x: x[3])[-N:]])

def select_best_chromosomes(population,N):
    return [i for i in sorted(population, key=lambda x: x[3])[-N:]]

def worst_chromosomes(population,N):
    worst = [i for i in sorted(population, key=lambda x: x[3])[:N]]
    
    for i in worst:
        population.remove(i)
    return population

def evolve(j,islands,iteration,m_iter,N,strategies,n,b,stop_loss,take_profit,allocated_capital,K,r_cross,r_mut,r_inv):
    """evolve each island per generation"""
    island = islands[j]
    for chromosome in island:
        fitness_function(chromosome,strategies,n,b,stop_loss,take_profit,allocated_capital,K)
    tempPopu  = selection(island)
    children = []
    #Crossover
    for i in range(0, len(tempPopu)-1, 2):
            # get selected parents in pairs
            parent1,parent2 = tempPopu[i],tempPopu[i+1]
            #crossover and mutation and inversion 
            child1,child2 = crossover(parent1,parent2,r_cross)
            child1 = mutation(child1, r_mut)
            child2 = mutation(child2, r_mut)
            child1 = inversion(child1, r_inv)
            child2 = inversion(child2, r_inv)
            children.append(child1)
            children.append(child2)
    island = children
    ## migration
    if iteration % m_iter ==0:
        
        island = worst_chromosomes(island,N)
        
        for index, item in enumerate(islands):
            if index == len(islands) - 1:
                receive_individuals = select_best_chromosomes(islands[0],N)
            else:
                receive_individuals = select_best_chromosomes(islands[index+1],N)
        for chromosome in receive_individuals:
            island.append(chromosome)
    
    return island   


def island_gga(num_island,num_iter,data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital):
    num_weight = (K*2) + 1
    #intiate population
    islands = []
    for i in range(num_island):
        islands.append(init_population(pSize,n,b,K,num_weight,strategies))
    #evolve
    for iteration in range(num_iter):
        pool =mp.Pool(processes=num_island)
        evolve_results = [pool.apply(evolve, args=(j,islands,iteration,m_iter,N,strategies,n,b,stop_loss,take_profit,allocated_capital,K,r_cross,r_mut,r_inv))for j in range(num_island)]
        pool.close()
        pool.join()
        islands = evolve_results
       #migration 
    # Return the best individual from each island
    best_individuals = []
    processes = []
    result_queues = []
    for i in range(num_island):
        result_queue =mp.Queue()
        process =mp.Process(target=best_chromosomes, args=(islands[i],1,result_queue))
        process.start()
        processes.append(process)
        result_queues.append(result_queue)
    for process in processes:
        process.join()
    for process in result_queues:
        best_individuals.extend(process.get())
    print(len(best_individuals))
    globalBest = select_best_chromosomes(best_individuals,1)
    return globalBest

def main():
    data,strategies =  generate_candidate_trading_signals(aapl.copy())
    num_island = 3 # number of islands
    m_iter =  5 # migration interval
    N = 1 # migration rate
    K = 3 # number of grps
    pSize = 5 # pop size
    r_cross = 0.8 # cross over rate
    r_mut = 0.01 # mutation rate
    r_inv = 0.3 # inversion rate
    num_iter = 10 # number of generations
    n = 4 # number of SL  bits
    b = 4 # number of TP bits
    stop_loss = -0.15
    take_profit = 0.15
    allocated_capital=1000
    results = island_gga(num_island,num_iter,data,strategies,pSize,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)
    
    print(results)
    

if __name__ == "__main__":
   main()
   #print("cool")


        

In [ ]:
def chromosome2vector(population):
    migration_pool = []
    for chromosome in population:
        chromosome_vector = []
        for sltp in chromosome[0]:
            chromosome_vector.append(sltp)
        for grp in chromosome[1]:
            for ts in grp:
                chromosome_vector.append(ts)
        for w in chromosome[2]:
            chromosome_vector.append(w)
        chromosome_vector.append(chromosome[3])
        migration_pool.append(chromosome_vector)
    
    return migration_pool
#chromosome2vector()

In [ ]:
##ISLAND GGA MULTIKUTI ALGORITHM
import time
def lowest(accepted_pool,best):
    """lowest chromosome to replace"""
    lowest = accepted_pool[0]
    score = hamming_distance(lowest,best[0])
    for i in range(len(accepted_pool)):
        for j in range(len(best)):
            dist = hamming_distance(accepted_pool[i],best[j])
            if dist < score:
                lowest=accepted_pool[i]
                score = dist
    return lowest, score



def multikuti_migration(population,immigrants,N,M):
    best = best_chromosomes(population,N) # pool to select the most different chromosomes from. 
    accepted_immigrants = immigrants[:M] # selected chromosomes which are most different chromosomes. 
    lowest_immigrant,dist_score = lowest(accepted_immigrants,best)
    for i in range(len(immigrants)):
        for j in range(i, len(best)):
            dist = hamming_distance(immigrants[i], best[j])
            if dist > dist_score:
                if immigrants[i] not in accepted_immigrants:
                    accepted_immigrants.remove(lowest_immigrant)
                    accepted_immigrants.append(immigrants[i])
                    dist_score = dist
                    lowest_immigrant = immigrants[i]
                else:
                    dist_score = dist
                    lowest_immigrant = immigrants[i]
    return accepted_immigrants
    
def island_gga(populations,island,immigrants,iteration,data,strategies,m_iter,N,M,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital):
    num_weight = (K*2) + 1
    #best = populations[island][:N]
    
    for chromosome in populations[island]:
        fitness_function(chromosome,strategies,n,b,stop_loss,take_profit,allocated_capital,K)
    tempPopu  = selection(populations[island])
    children = []
    #Crossover
    for i in range(0, len(tempPopu)-1, 2):

            # get selected parents in pairs
            parent1,parent2 = tempPopu[i],tempPopu[i+1]
            #crossover and mutation and inversion 
            child1,child2 = crossover(parent1,parent2,r_cross)
            child1 = mutation(child1, r_mut)
            child2 = mutation(child2, r_mut)
            child1 = inversion(child1, r_inv)
            child2 = inversion(child2, r_inv)
            children.append(child1)
            children.append(child2)
    populations[island] = children
#migration 
    if iteration % m_iter ==0:
        best = best_chromosomes(populations[island],N)# get best
        worst_chromosomes(populations[island],M) # delete worst chromosomes
        #ring based multikuti topology migration
        if immigrants:
            accepted_immigrants = multikuti_migration(populations[island],immigrants,N,M)
            for i in accepted_immigrants:
                # add the most diverse immigrants
                populations[island].append(i)

        #add this island best chromosomes to next island
        for chromosome in best:
            immigrants.append(chromosome)

    return True

def main():
    #island_gga(queue,data,strategies,n_islands,m_iter,N,K,pSize,r_cross,r_mut,r_inv,n_iter,n,b,stop_loss,take_profit,allocated_capital)
    data,strategies =  generate_candidate_trading_signals(train.copy())
    #lock = Lock()
    n_islands = int(input("islands: "))
    m_iter = int(input("Migration Interval: "))
    N = int(input("Migration Rate: "))
    K = int(input("K: "))
    pSize = int(input("popSize: "))
    r_cross = float(input("Cross over rate: "))
    r_mut = float(input("Mutation Rate: "))
    r_inv = float(input("Inversion Rate: "))
    n_iter = int(input("Number of Generations: "))
    n = int(input("SL: "))
    b = int(input("TP: "))
    stop_loss = float(input("Stop  Loss: "))
    take_profit = float(input("Take Profit: "))
    allocated_capital= float(input("Capital: "))
    num_weight = (K*2) + 1
    gga_start = time.time()
    gga_r = gga(data,K,pSize,strategies,r_cross,r_mut,r_inv,n_iter,n,b,stop_loss,take_profit,allocated_capital)
    igga_start = time.time()
    populations  = {i: init_population(pSize,n,b,K,num_weight,strategies) for i in range(n_islands)}#
    immigrants = []
    for iteration in range(n_iter):
        for island in populations:
            island_gga(populations,island,immigrants,iteration,data,strategies,m_iter,N,K,r_cross,r_mut,r_inv,n,b,stop_loss,take_profit,allocated_capital)
            
   
    results = best_chromosomes(immigrants,1)
    igga_end = time.time()
    print(results)

if __name__ == "__main__":
    main()

        